In [162]:
from glob import glob
import vdp
import re
import numpy as np


from vdp.vocabulary import FOSort, FOFunction
from vdp.fomodel import FOElement, FOModel
from vdp.exceptions import PartialInterpretationException


get_name = re.compile(r"Enter Image Path: .+\/([^/]+).jpg: Predicted in \d*\.?\d* seconds.")
split_bb = re.compile(r"Left=(\d+), Top=(\d+), Right=(\d+), Bottom=(\d+)")

In [163]:
files = glob("./vdp_smt/*/*/*train_out.txt")

In [164]:
file = files[0]

In [165]:
lines = []
with open(file, 'r') as f:
    lines = [line.replace("\n", "") for line in f.readlines()]

mask = [("Image Path" in line) for line in lines]
images = [get_name.match(line).group(1) for idx, line in enumerate(lines) if mask[idx] if "Predicted" in line]
labels = [(np.where(mask[:idx+1])[0][-1], line) for idx, line in enumerate(lines) if not mask[idx] ]


In [167]:
predicts = {}

for i in range(len(lines)):
    if mask[i] and "Predicted" in lines[i]:
        img_name = get_name.match(lines[i]).group(1)
        next_i = i + np.argmax(mask[i+1:])
        predicts[img_name] = list(zip(lines[i+1:next_i:2], lines[i+2:next_i:2]))
        

In [383]:
img = predicts['train_e']
img_params = [(*item[0].split(": "), *split_bb.findall(item[1].split(": ")[-1])) for item in img ]

In [384]:
centroid = lambda bb : ((int(bb[0]) + int(bb[2])) / 2, (int(bb[1]) + int(bb[3])) / 2)


def relate(c1, c2): # where is c1 w.r.t c2
    x1, y1 = c1
    x2, y2 = c2
    
    return ["right" if x1 >= x2 else "left", "above" if y1 >= y2 else "below"]

relations = {}
for label1, pred1, bb1 in img_params:
    for label2, pred2, bb2 in img_params:
        if label1 != label2:
            c1 = centroid(bb1)
            c2 = centroid(bb2)
            relations[(label1, label2)] = relate(c1, c2)
        else:
            relations[(label1, label2)] = ['labelOf']
            
labels, preds, bbs = zip(*img_params)

relations


{('bed', 'bed'): ['labelOf'],
 ('bed', 'sofa'): ['right', 'above'],
 ('bed', 'cat'): ['left', 'above'],
 ('sofa', 'bed'): ['left', 'below'],
 ('sofa', 'sofa'): ['labelOf'],
 ('sofa', 'cat'): ['left', 'above'],
 ('cat', 'bed'): ['right', 'below'],
 ('cat', 'sofa'): ['right', 'below'],
 ('cat', 'cat'): ['labelOf']}

In [431]:
vdpobject = FOSort('Object')
vdplabel = FOSort('Label')
vdpbool = FOSort('Bool')


In [432]:
vdpconstants = dict()
for label in labels:
    if label not in vdpconstants:
        vdpconstants[label] = FOFunction(label, tuple([vdplabel]))
    
    
unique_labels = list(vdpconstants.keys()) # @TODO REPLACE!!!!


In [461]:
# on = FOFunction('On', (vdpobject, vdpobject, vdpbool))
relation_labels = ['left', 'right', 'above', 'below']
vdprelations = dict()

# for rl in relation_labels:
#     vdprelations[rl] = FOFunction(rl, (vdpobject, vdpobject, vdpbool))


vdprelations['labelOf'] = FOFunction('labelOf', (vdpobject, vdplabel, vdpbool))
vdprelations

{'labelOf': <vdp.vocabulary.FOFunction at 0x117cd7dd0>}

In [462]:
vdpvocabulary = (set([vdpobject, vdplabel]), set(list(vdpconstants.values()) + list(vdprelations.values())))


In [463]:
vdpobjects = {label : FOElement(label + "_object", vdpobject) for label in labels}
vdplabels = {label : FOElement(label + "_label", vdplabel) for label in unique_labels}

vdpelements = {'Object': set(vdpobjects.values()), 'Label': set(vdplabels.values())}

In [478]:
labelconst_interpretation = {const : vdplabels[label] for label, const in zip(unique_labels, list(constants.values()))}
labelconst_interpretation

{<vdp.vocabulary.FOFunction at 0x117c35850>: <vdp.fomodel.FOElement at 0x117c59150>,
 <vdp.vocabulary.FOFunction at 0x117c354d0>: <vdp.fomodel.FOElement at 0x117c59d50>,
 <vdp.vocabulary.FOFunction at 0x117c35710>: <vdp.fomodel.FOElement at 0x117c59b10>}

In [479]:

    

rel_interpretation = dict()
# for rel_sym, rel in vdprelations.items():
#     if 'labelOf' in rel_sym:
#         rel_interpretation[rel] = {(obj, lab) : rel_sym in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}
#     else:
#         rel_interpretation[rel] = {(obj1, obj2) : rel_sym in relations[(obj1_sym, obj2_sym)]  for obj1_sym, obj1 in vdpobjects.items() for obj2_sym, obj2 in vdpobjects.items() if obj1_sym != obj2_sym}
        
        
rel_interpretation[vdprelations['labelOf']] = {(obj, lab) : 'labelOf' in relations[(obj_sym, lab_sym)]  for obj_sym, obj in vdpobjects.items() for lab_sym, lab in vdplabels.items()}


rel_interpretation

{<vdp.vocabulary.FOFunction at 0x117cd7dd0>: {(<vdp.fomodel.FOElement at 0x117c59690>,
   <vdp.fomodel.FOElement at 0x117c59150>): True,
  (<vdp.fomodel.FOElement at 0x117c59690>,
   <vdp.fomodel.FOElement at 0x117c59d50>): False,
  (<vdp.fomodel.FOElement at 0x117c59690>,
   <vdp.fomodel.FOElement at 0x117c59b10>): False,
  (<vdp.fomodel.FOElement at 0x117c59050>,
   <vdp.fomodel.FOElement at 0x117c59150>): False,
  (<vdp.fomodel.FOElement at 0x117c59050>,
   <vdp.fomodel.FOElement at 0x117c59d50>): True,
  (<vdp.fomodel.FOElement at 0x117c59050>,
   <vdp.fomodel.FOElement at 0x117c59b10>): False,
  (<vdp.fomodel.FOElement at 0x117c59a50>,
   <vdp.fomodel.FOElement at 0x117c59150>): False,
  (<vdp.fomodel.FOElement at 0x117c59a50>,
   <vdp.fomodel.FOElement at 0x117c59d50>): False,
  (<vdp.fomodel.FOElement at 0x117c59a50>,
   <vdp.fomodel.FOElement at 0x117c59b10>): True}}

In [485]:
interpretation = {**labelconst_interpretation, **rel_interpretation}


In [484]:
model = FOModel(vocabulary, elements, interpretation)


In [483]:
tst = FOElement('x', vdpobject)
model.interpret(term=[vdprelations['labelOf'], [tst], [vdpconstants['cat']]], interpretation_extension={tst: vdpobjects['cat']})

True